# PLAN




    * Steps 1-10

    Summary of what we have done so far. Just check the headers to see if you missed  anything.

    * Step 11 
    
    Cross validation with the training data.
    
    * Step 12 

    Testing the model on test data.
    
    * Step 13

    Extending the prediction to the future.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from statsmodels.tsa.ar_model ar_select_order

# Set figure size to (14,6)
plt.rcParams['figure.figsize'] = (14,6)

# Step 1 - Load the Data

In [ ]:
flights = pd.read_csv('data/flights_train.csv', index_col=0, parse_dates=True)
flights.head()

In [ ]:
# Inspect the size of the data
flights.shape # 11 years of monthly data

In [ ]:
flights

# Plot the data

In [ ]:
def plot_flights(df, title='Monthly Passenger Numbers in 1000 over Time', ylim=True):
    '''
    Custom plotting function for plotting the flights dataset
    
    Parameters
    ----------
    df : pd.DataFrame
        The data to plot.
    title : str
        The title of the plot
    ylim : bool
        Whether to fix the minimum value of y; defalut is True
    
    Returns
    -------
    Plots the data
    '''
    df.plot()
    plt.title(title)
    plt.ylabel('# of Passengers in 1000')
    if ylim:
        plt.ylim(ymin=0)
    plt.show()

In [ ]:
plot_flights(flights)

In [ ]:
flights

# Step 2 - Clean the Data

Fortunately we do not have to do that in case of the flights data. The steps for this for the temperature data are summarised in the AR notebook.

# Step 3 - Extract the Timestep and the Seasonal Dummies for the whole Dataset

In [ ]:
# Create a timestep variable
flights['timestep'] = list(range(len(flights)))
flights

In [ ]:
# Create the seasonal dummies
seasonal_dummies = pd.get_dummies(flights.index.month,
                                  prefix='month',
                                  drop_first=True).set_index(flights.index)

flights = flights.join(seasonal_dummies)
flights.head()

# 4) Train-Test-Split

Fortunately not necessary for the flights data.

# 5) Model the Trend_Seasonal model

In [ ]:
# Define X and y
X = flights.drop(columns=['passengers'])
y = flights['passengers']

In [ ]:
y

In [ ]:
# Create and fit the model
m = LinearRegression()
m.fit(X, y)

In [ ]:
# Create a new column with the predictions of the trend_seasonal model
flights['trend_seasonal'] = m.predict(X)
flights.head()

# Plot the original data and preliminary model

In [ ]:
plot_flights(flights[['passengers', 'trend_seasonal']])

# 6) - Extract the remainder

In [ ]:
# We want to extract the part of the model that the trend_seasonal is not able to explain
flights['remainder'] = flights['passengers'] - flights['trend_seasonal']

In [ ]:
plot_flights(flights['remainder'], 
             title='Remainder after modelling trend and seasonality', 
             ylim=False)

In [ ]:
flights["remainder"].std(), flights["passengers"].std()

# 7) - Inspect the remainder to decide how many lags to include


In [ ]:
selected_order = ar_select_order(flights['remainder'], maxlag = 12)
selected_order.ar_lags

# 8) - Add the lags of the remainder to the training data

In [ ]:
flights['lag1'] = flights['remainder'].shift(1)
flights.dropna(inplace=True)
flights.head()

In [ ]:
flights

# 9) Run the full model

In [ ]:
# Assign X
X_full = flights.drop(columns=['passengers', 'trend_seasonal', 'remainder'])
y_full = flights['passengers']

Make sure you do not modify the original flights data when dropping columns etc. so we can keep using it in the next steps whenever necessary.

In [ ]:
X_full.head()

In [ ]:
m_full = LinearRegression()
m_full.fit(X_full, y_full)

In [ ]:
# Create a new predictions column
flights['predictions_full_model'] = m_full.predict(X_full)

In [ ]:
flights.tail()

So the timestep, monthly dummies and the lag of the remainder actually gave us the predictions for the actual data (not just the remainder).

# 10) - Plot the prediction vs passengers for the training data

In [ ]:
plot_flights(flights.loc['1958-12-01':'1959-12-01',['passengers', 'trend_seasonal', 'predictions_full_model']])

Coefficients of the AR equation's terms:

In [ ]:
#m_full.coef_
m_full.intercept_

In [ ]:
pd.DataFrame(m_full.coef_.reshape(1,13), columns=X_full.columns)

---------------
---------------

This was what we did until this morning, exclusively with the training data. 

# Is this model good?

# 11) - Evaluate our model

We want to understand how good our model would work on data it has not been trained on. We can get an estimate of that by using cross-validation.

Cross-validation so far:



Cross-validation for time series:



In [ ]:
X_full

In [ ]:
# Create a TimeSeriesSplit object
ts_split = TimeSeriesSplit(n_splits=5)

In [ ]:
# Split the training data into folds
for i, (train_index, validation_index) in enumerate(ts_split.split(X_full, y_full)):
    print(f"""The training data for the {i+1}th iteration are the observations steps 
    {train_index}""")
    print(f"""The validation data for the {i+1}th iteration are the observations steps
    {validation_index}""")
    print('\n')
    

In [ ]:
# Create the time series split
time_series_split = ts_split.split(X_full, y_full) 

In [ ]:
# Do the cross validation: Remember these are the 'test scores' in the training data.
result = cross_val_score(estimator=m_full, 
                         X=X_full, y=y_full,
                         cv=time_series_split)
result

In [ ]:
result.mean()

# 12) - Test your model

We finally use the test data. 

In [ ]:
flights_test = pd.read_csv('data/flights_test.csv', index_col=0, parse_dates=True)
flights_test.head()

In [ ]:
flights_test

First, the transformations necessary for the trend-seasonal model test: 

We will need to define the timestep using the last timestep of the training data (unless you split after adding the timestep and dummies). 

In [ ]:
# Get last timestep of the training data
last_train_timestep = flights['timestep'][-1]

In [ ]:
# Create a timestep for the whole test data
flights_test['timestep'] = list(range(last_train_timestep + 1, 
                            last_train_timestep + len(flights_test) + 1))
flights_test.head()

Now let's create the dummies for the seasonal component of the test.

In [ ]:
seasonal_dummies = pd.get_dummies(flights_test.index.month, 
                                  prefix='month', 
                                  drop_first=True).set_index(flights_test.index)

flights_test = flights_test.join(seasonal_dummies)
flights_test.head()

Let's select X and y.

In [ ]:
X_test = flights_test.drop(columns=['passengers'])

In [ ]:
X_test

Predict trend and seasonality for the test using **m** (which is trend_seasonal model for the training).

In [ ]:
flights_test['trend_seasonal'] = m.predict(X_test)
flights_test.head()

In [ ]:
plot_flights(flights_test[['passengers', 'trend_seasonal']], ylim=False)

Calculate the remainder for the test set.

In [ ]:
flights_test['remainder'] = flights_test['passengers'] - flights_test['trend_seasonal']


And finally the lag for the remainder as our additional feature for the AR model.

In [ ]:
# Create the lagged variable
flights_test['lag1'] = flights_test['remainder'].shift(1)

In [ ]:
# Assign X_full
X_full = flights_test.drop(columns=['passengers', 'trend_seasonal', 'remainder'])

In [ ]:
X_full.head() # Contains a NaN for the first value of lag1

Filling in the missing value from the beginning of the test lag.

In [ ]:
X_full.loc['1960-01-01', 'lag1'] = flights.loc['1959-12-01', 'remainder']

In [ ]:
# Create the predictions
flights_test['predictions_full_model'] = m_full.predict(X_full)

In [ ]:
plot_flights(flights_test[['passengers', 'trend_seasonal', 'predictions_full_model']], ylim=False)

In [ ]:
# Create the complete dataset and plot it
flights_full = flights[['passengers', 'trend_seasonal', 'predictions_full_model']].append(flights_test[['passengers', 'trend_seasonal', 'predictions_full_model']])

In [ ]:
flights_full.head()

In [ ]:
plot_flights(flights_full)

In [ ]:
print(f"""
{m_full.score(X_full, flights_test['passengers'])}
{m.score(X_test, flights_test['passengers'])}
""")


In [ ]:
X_full

In [ ]:
flights_test

# 13) - Predict the future

So far we have just predicted data that we already had (train and test). We have not actually made any predictions for the future.

In [ ]:
flights_test.head()

In [ ]:
# Combine train and test data
flights_combined = flights.append(flights_test)
flights_combined

In [ ]:
# Re-train the model on the whole dataset
X_combined = flights_combined.drop(columns=['passengers', 'trend_seasonal', 'remainder', 'predictions_full_model'])
y_combined = flights_combined['passengers']

In [ ]:
flights_combined

In [ ]:
X_combined

Fill in the missing value from the beginning of the test set.

In [ ]:
X_combined.loc['1960-01-01', 'lag1'] = flights.loc['1959-12-01', 'remainder']

In [ ]:
X_combined.tail(1)

In [ ]:
m_combined = LinearRegression()
m_combined.fit(X_combined, y_combined)

### We are going to create a single future step data.

In [ ]:
flights_combined.tail(1)

In [ ]:
# What is the first date in the future? --> 1961-01-01
timestep = flights_combined['timestep'].max() + 1
months = [0] * 11
lag = flights_combined.loc['1960-12-01', 'remainder']

In [ ]:
timestep, months, lag

In [ ]:
X_future = [timestep]

X_future

In [ ]:
X_future.extend(months)
X_future

Check what would have happened above if you had used append instead of extend.

In [ ]:
X_future.append(lag)
X_future

In [ ]:
X_future = pd.DataFrame([X_future])
X_future.columns = X_combined.columns

X_future

In [ ]:
# Prediction for 1961-01-01
# m_full.predict(X_future) is somehow a better idea.
m_combined.predict(X_future)

In [ ]:
X_future.shape

In [ ]:
# How does this look like for 1961-02-01? So one more step to the future.
timestep = flights_combined['timestep'].max() + 2
months = [1] + [0]*10
lag = 0 
# This is too far in the future to calculate the lag.

In [ ]:
X_future_2 = pd.DataFrame([[timestep] + months + [lag]])
X_future_2.columns = X_combined.columns
X_future_2

In [ ]:
# Prediction for 1961-02-01

m_combined.predict(X_future_2)

The prediction for 1961-02-01 was just the prediction of trend and seasonality.